# Capítulo II: Retrieval

### Document loaders

In [4]:
#!pip install unstructured pdf2image pdfminer pdfminer-six

In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain.document_loaders import OnlinePDFLoader

In [2]:
#Este es un ejemplo de carga de un documento pdf que se encuentra de forma online. Existen múltiples opciones para documentos con ubicación local
loader = OnlinePDFLoader("https://portal.beneficiosestudiantiles.cl/sites/default/files/decreto-97_09-oct-2013.pdf")

In [8]:
data = loader.load()

In [16]:
data[0]

Document(page_content='Decreto 97, EDUCACIÓN (2013)\n\nDecreto 97 REGLAMENTA EL PROGRAMA DE BECAS DE EDUCACIÓN SUPERIOR\n\nMINISTERIO DE EDUCACIÓN\n\nFecha Publicación: 09-OCT-2013 | Fecha Promulgación: 22-FEB-2013 Tipo Versión: Última Versión De : 02-MAR-2023\n\nUltima Modificación: 02-MAR-2023 Decreto 221\n\nUrl Corta: https://bcn.cl/3c1l8\n\nREGLAMENTA EL PROGRAMA DE BECAS DE EDUCACIÓN SUPERIOR\n\nNúm. 97.- Santiago, 22 de febrero de 2013.- Considerando:\n\nQue la Ley Nº 20.641, de Presupuestos del Sector Público año 2013, en su Partida 09, Capítulo 01, Programa 30, Subtítulo 24, Ítem 03, Asignación 200, Glosa 03 contempla recursos para becas de educación superior. Que, según dicha asignación, el referido Programa se ejecutará de acuerdo al decreto Nº 116, de 2012, del Ministerio de Educación, que modifica el decreto Nº 337, de 2010, que reglamenta el Programa de Becas de Educación Superior, año 2010. Que, a efectos de dar cumplimiento a lo dispuesto en la ley Nº 20.641, resulta una

### Document Transformer

Cuando se trata de documentos genéricos, se sugiere que esta transformación es la más adecuada. Ya que utiliza la lista ["\n\n", "\n", " ", ""], esto tiene el efecto de intentar mantener todos los párrafos (y luego las oraciones, y después las palabras) juntos el mayor tiempo posible, ya que genéricamente parecerían ser las piezas de texto semánticamente relacionadas más fuertemente.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

In [17]:
all_splits = text_splitter.split_documents([data[0]])

In [23]:
#También es posible particionar por token, usando tiktoken
#!pip install tiktoken
from langchain.text_splitter import TokenTextSplitter

In [59]:
token_text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)

In [60]:
all_token_splits = token_text_splitter.split_documents([data[0]])

### Text Embedding Model

In [32]:
#En este ejercicio usaremos el modelo de OpenAI para Embeddings
from langchain.embeddings import OpenAIEmbeddings

In [35]:
#Instanciamos el modelo de Embeddings
embeddings_model = OpenAIEmbeddings()

In [43]:
#Se aplica el modelo a una muestra de 5 chunks
embeddings = embeddings_model.embed_documents(
    [x.page_content for x in all_splits[:5]]
)

In [46]:
#Dimensiones del tensor: total de vectores, largo de cada vector
len(embeddings), len(embeddings[0])

(5, 1536)

### Vector Store

Si bien es posible almacenar temporalmente los embeddings en caché, para fines de esta sesión usaremos una base vectorial. En este caso, será ChromaDB

In [47]:
from langchain.vectorstores import Chroma

In [62]:
#Construcción de la bbdd
db = Chroma.from_documents(all_token_splits, embeddings_model)

In [63]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
docs = db.similarity_search(query)

In [66]:
print(docs[0].page_content)

Sólo podrán acceder a esta beca los estudiantes que EDUCACION provienen de los hogares


### Retrievers

Un retriever es una interfaz que devuelve documentos a partir de una consulta no estructurada. Es más general que un vector database. Un retriever no necesita poder almacenar documentos, solo devolverlos (o recuperarlos). Los vector databases pueden usarse como la base de un retriever, pero también existen otros tipos de retriever.

Para entender retrievers, usaremos el ejercicio de preguntas-respuestas sobre documentos. Las etapas generales que debe cumplir este proceso son:
* Crear un índice
* Crear un retriever desde ese índice
* Crear una cadena de pregunta-respuesta
* ¡A hacer preguntas!

In [73]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA #<--- Por ahora un golazo :)
from langchain.llms import OpenAI

In [68]:
loader = OnlinePDFLoader("https://portal.beneficiosestudiantiles.cl/sites/default/files/decreto-97_09-oct-2013.pdf")
documents = loader.load()

In [69]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(documents)

In [70]:
embeddings = OpenAIEmbeddings()

In [71]:
db = Chroma.from_documents(texts, embeddings)

In [72]:
#Ya creado el indice, hacemos la interfaz para el retriever
retriever = db.as_retriever()

In [74]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever) #<--- acá se materializa el golazo

In [75]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
qa.run(query)

' No, los estudiantes solo podrán acceder a la beca si pertenecen a los primeros siete deciles de menores ingresos de la población del país.'

In [76]:
query = "Quiero acceder a gratuidad, que condiciones debo cumplir?"
qa.run(query)

' Para postular a la gratuidad, los estudiantes deberán cumplir los requisitos señalados en el artículo 44 bis del decreto 77, EDUCACIÓN, y provienen de los hogares pertenecientes a los siete deciles de menores ingresos de la población del país, y que se matriculen en una institución de educación superior que cuente con acreditación institucional vigente de al menos cuatro años, al 31 de diciembre del año anterior a la asignación del beneficio, conforme a la ley Nº 20.129. Además, deben renunciar a la beca mediante el formato dispuesto por la Subsecretaría para este efecto, dentro del proceso de ciamiento institucional para la gratuidad establecido en la ley Nº 21.091.'

In [77]:
#Para lo anterior, LangChain tiene una solución rápida
from langchain.indexes import VectorstoreIndexCreator

In [78]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [82]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
index.query(query)

' No, solo los estudiantes que provienen de los primeros siete deciles de menores ingresos de la población del país pueden acceder a esta beca.'